In [2]:
import pandas as pd
import pubchempy as pcp
from pybel import readstring, readfile
import requests
import time
from pychem.pychem import *
from rdkit import Chem
from StringIO import StringIO
from glob import glob
from subprocess import check_output, CalledProcessError

DATA = '../data/version-3/'

In [3]:
chemicals = pd.read_csv(DATA+'food-chemical.tsv', sep='\t')
print("Unique chemicals: ", len(set(chemicals['pubchem-id'])))

('Unique chemicals: ', 6992)


### Get PubChem SDF Files

### Generate ChemoPy properties

In [4]:
# Get properties.
def generate_properties(mol):
    props = {}
    try:
        props.update(constitution.GetConstitutional(mol))
        props.update(connectivity.GetConnectivity(mol))
        props.update(kappa.GetKappa(mol))
        props.update(bcut.GetBurden(mol))
        props.update(estate.GetEstate(mol))
        props.update(basak.Getbasak(mol))
        props.update(moran.GetMoranAuto(mol))
        props.update(geary.GetGearyAuto(mol))
        props.update(molproperty.GetMolecularProperty(mol))
        props.update(charge.GetCharge(mol))
        props.update(moe.GetMOE(mol))
    except:
        pass

    return props



In [16]:
sdf_files = [readfile('sdf', f).next() for f in glob(DATA+'sdf-files/*.sdf')]
properties = list()
details = list()
covered = set()

In [25]:
for i, mol in enumerate(sdf_files):
    if i not in covered:
        molecule = Chem.MolFromMolBlock(mol.write('sdf'))
        properties.append(generate_properties(molecule))
        details.append(mol.data)

    if i % 100 == 0:
        print("Completed: %i" % i)

# Save pubchem info
pubchem_info = pd.DataFrame([dict(d) for d in details])
pubchem_info.to_csv(DATA+'pubchem_details.tsv', sep='\t', encoding='utf-8', index=None)

# Save chemical properties
properties = pd.DataFrame(properties)
properties.to_csv(DATA+'properties.tsv', sep='\t', encoding='utf-8', index=None)

Completed: 0


/Users/rudraksh/miniconda3/envs/bsserver/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/rudraksh/miniconda3/envs/bsserver/lib/python2.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Completed: 100
Completed: 200
Completed: 300
Completed: 400
Completed: 500
Completed: 600


/Users/rudraksh/miniconda3/envs/bsserver/lib/python2.7/site-packages/pychem/charge.py:448: RuntimeWarning: invalid value encountered in greater
  return round(numpy.mean(cc[cc>0]),3)


Completed: 700
Completed: 800
Completed: 900


/Users/rudraksh/miniconda3/envs/bsserver/lib/python2.7/site-packages/pychem/charge.py:420: RuntimeWarning: invalid value encountered in greater
  return round(sum(cc[cc>0]),3)
/Users/rudraksh/miniconda3/envs/bsserver/lib/python2.7/site-packages/pychem/charge.py:477: RuntimeWarning: invalid value encountered in less
  return round(sum(cc[cc<0]),3)
/Users/rudraksh/miniconda3/envs/bsserver/lib/python2.7/site-packages/pychem/charge.py:626: RuntimeWarning: invalid value encountered in less
  if sum(cc[cc<0])==0:
/Users/rudraksh/miniconda3/envs/bsserver/lib/python2.7/site-packages/pychem/charge.py:593: RuntimeWarning: invalid value encountered in greater
  if sum(cc[cc>0])==0:
/Users/rudraksh/miniconda3/envs/bsserver/lib/python2.7/site-packages/pychem/charge.py:506: RuntimeWarning: invalid value encountered in less
  return round(numpy.mean(cc[cc<0]),3)


Completed: 1000
Completed: 1100
Completed: 1200
Completed: 1300
Completed: 1400
Completed: 1500
Completed: 1600
Completed: 1700
Completed: 1800
Completed: 1900
Completed: 2000
Completed: 2100
Completed: 2200
Completed: 2300
Completed: 2400
Completed: 2500
Completed: 2600
Completed: 2700
Completed: 2800
Completed: 2900


/Users/rudraksh/miniconda3/envs/bsserver/lib/python2.7/site-packages/pychem/connectivity.py:546: RuntimeWarning: invalid value encountered in sqrt
  res=sum(numpy.sqrt(1./deltas))


Completed: 3000
Completed: 3100
Completed: 3200
Completed: 3300
Completed: 3400
Completed: 3500
Completed: 3600
Completed: 3700
Completed: 3800
Completed: 3900
Completed: 4000
Completed: 4100
Completed: 4200
Completed: 4300
Completed: 4400
Completed: 4500
Completed: 4600
Completed: 4700
Completed: 4800
Completed: 4900
Completed: 5000
Completed: 5100
Completed: 5200
Completed: 5300
Completed: 5400
Completed: 5500
Completed: 5600
Completed: 5700
Completed: 5800
Completed: 5900
Completed: 6000
Completed: 6100
Completed: 6200
Completed: 6300
Completed: 6400
Completed: 6500
Completed: 6600
Completed: 6700
Completed: 6800
Completed: 6900


/Users/rudraksh/miniconda3/envs/bsserver/lib/python2.7/site-packages/pychem/connectivity.py:991: RuntimeWarning: divide by zero encountered in double_scalars
  accum=accum+1./numpy.sqrt(deltas1.prod())
/Users/rudraksh/miniconda3/envs/bsserver/lib/python2.7/site-packages/pychem/connectivity.py:929: RuntimeWarning: divide by zero encountered in double_scalars
  accum=accum+1./numpy.sqrt(deltas1.prod())
/Users/rudraksh/miniconda3/envs/bsserver/lib/python2.7/site-packages/pychem/connectivity.py:959: RuntimeWarning: divide by zero encountered in double_scalars
  accum=accum+1./numpy.sqrt(deltas1.prod())
/Users/rudraksh/miniconda3/envs/bsserver/lib/python2.7/site-packages/pychem/connectivity.py:1010: RuntimeWarning: invalid value encountered in double_scalars
  return abs(CalculateChiv3c(mol)-CalculateChiv4pc(mol))


### Get common names from PubChem

In [32]:
pubchem_info = pd.read_csv(DATA+'pubchem_details.tsv', sep='\t', encoding='utf-8')

In [33]:
pchem2synonyms = dict()
pchems = list(set(pubchem_info[u'PUBCHEM_COMPOUND_CID']) - {0})
completed = set()

In [34]:
for i in range(0, len(pchems), 100):

    if i not in completed:
        subs_pchems = pchems[i: i+100] 

        for c in pcp.get_synonyms(subs_pchems):
            try:
                pchem2synonyms[c['CID']] = '|'.join(c['Synonym'])
            except KeyError:
                continue
                
        completed.add(i)
    else:
        continue

    print("Completed: ", len(pchem2synonyms))

('Completed: ', 96)
('Completed: ', 189)
('Completed: ', 289)
('Completed: ', 389)
('Completed: ', 482)
('Completed: ', 577)
('Completed: ', 676)
('Completed: ', 770)
('Completed: ', 861)
('Completed: ', 949)
('Completed: ', 1040)
('Completed: ', 1130)
('Completed: ', 1219)
('Completed: ', 1317)
('Completed: ', 1412)
('Completed: ', 1509)
('Completed: ', 1606)
('Completed: ', 1706)
('Completed: ', 1801)
('Completed: ', 1897)
('Completed: ', 1996)
('Completed: ', 2092)
('Completed: ', 2192)
('Completed: ', 2289)
('Completed: ', 2383)
('Completed: ', 2478)
('Completed: ', 2573)
('Completed: ', 2669)
('Completed: ', 2767)
('Completed: ', 2865)
('Completed: ', 2954)
('Completed: ', 3050)
('Completed: ', 3144)
('Completed: ', 3238)
('Completed: ', 3335)
('Completed: ', 3430)
('Completed: ', 3523)
('Completed: ', 3617)
('Completed: ', 3716)
('Completed: ', 3807)
('Completed: ', 3902)
('Completed: ', 3998)
('Completed: ', 4092)
('Completed: ', 4186)
('Completed: ', 4284)
('Completed: ', 4379)

In [35]:
synonyms = list()

for pchem in pubchem_info['PUBCHEM_COMPOUND_CID']:
    try: synonyms.append(pchem2synonyms[pchem])
    except KeyError: synonyms.append('')
        
pubchem_info['synonyms'] = synonyms

### Save subset of all properties

In [36]:
properties = pd.read_csv(DATA+'properties.tsv', sep='\t', encoding='utf-8')

In [37]:
props_to_keep = ['Weight', 'nhyd', 'nring', 'nrot', 'ndonr', 'naccr', 'nta',
                 'naro', 'nhev', 'Hy', 'LogP']

chem_info = pd.concat([
    pubchem_info[['PUBCHEM_COMPOUND_CID', 'synonyms', 'PUBCHEM_OPENEYE_CAN_SMILES', 'PUBCHEM_IUPAC_INCHI',
              'PUBCHEM_IUPAC_INCHIKEY', 'PUBCHEM_IUPAC_NAME', 'PUBCHEM_OPENEYE_ISO_SMILES',
              'PUBCHEM_MOLECULAR_FORMULA']],
    properties[props_to_keep]],
    1)

### Functional groups

In [38]:
def generate_fg(smiles):
    mol = readstring('smi', smiles)
    mol.write('mol', 'temp.mol', overwrite=True)
    out = check_output(['checkmol', '-p', '-e', 'temp.mol'])

    # Process  and return output
    return ','.join([o.split(':')[0][1:] for o in out.split('\n') if o])

fgs = list()

In [39]:
for i, smi in enumerate(chem_info['PUBCHEM_OPENEYE_ISO_SMILES']):
    try:
        fg = generate_fg(smi)
        fgs.append([smi, fg])
    except CalledProcessError:
        fgs.append([smi, ''])
        
    if i % 100 == 0:
        print(i)
        
fgs_df = pd.DataFrame(fgs, columns=['smiles', 'functional_groups'])

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900


In [40]:
# Add to main dataframe
chem_info['functional_group_idx'] = fgs_df['functional_groups'].fillna('').tolist()

# Read mapping from functional group idx to name
fgidx_map = pd.read_csv('../../bittersweet-server/data/haider_fg_list.tsv',
                        sep='\t', encoding='utf-8', dtype=str)
fgidx_map = fgidx_map.set_index('id').to_dict()['functional_group']

chem_info['functional_group'] = chem_info['functional_group_idx'].map(
    lambda s: ', '.join([fgidx_map[fidx].capitalize() for fidx in s.split(',')]) if (s != '') else '')

In [48]:
import pickle as pkl

# Save list of functional groups present in db (for autocomplete)
unique_fgs = [fg for fglst in chem_info['functional_group'] .map(lambda s: s.split(', ')) for fg in fglst]
unique_fgs = list(set(unique_fgs))

pickle.dump(unique_fgs, open('../dietrx/static/unique_fgs.p', 'wb'))
pkl.dump(fgidx_map, open('../dietrx/static/fgid2name.p', 'wb'))
idxfg_map = {fg:id for id, fg in fgidx_map.items()}
pkl.dump(idxfg_map, open('../dietrx/static/fgname2id.p', 'wb'))

### Save chemical lexicon

In [44]:
chem_info['common_name'] = chem_info['synonyms'].fillna('').map(lambda s: unicode(s.split('|')[0]) if s else u'').apply(unicode.title)

In [52]:
# Change column names to match with db column names
chem_info.rename(columns={
    'PUBCHEM_COMPOUND_CID': 'pubchem_id',
    'Weight':'molecular_weight',
    'nhyd':'num_hydrogen_atoms',
    'nring':'num_rings',
    'nrot':'num_rotatablebonds',
    'ndonr':'hbd_count',
    'naccr':'hba_count',
    'nta':'num_atoms',
    'naro':'number_of_aromatic_bonds',
    'nhev':'num_heavy_atoms',
    'Hy':'hyrophilic_index',
    'LogP':'alogp',
    'PUBCHEM_OPENEYE_CAN_SMILES': 'canonical_smiles',
    'PUBCHEM_IUPAC_INCHI': 'inchi',
    'PUBCHEM_IUPAC_INCHIKEY': 'inchikey',
    'PUBCHEM_IUPAC_NAME': 'iupac_name',
    'PUBCHEM_OPENEYE_ISO_SMILES': 'isomeric_smiles',
    'PUBCHEM_MOLECULAR_FORMULA': 'molecular_formula'
}, inplace=True)

In [53]:
# Save to disk
chem_info.to_csv(DATA+'chemical-lexicon.tsv', sep='\t', index=None)

### Generate molecule images

In [26]:
from rdkit.Chem import MolFromSmiles, Draw
from rdkit.Chem.AllChem import Compute2DCoords
from shutil import copy

In [35]:
!rm {DATA/'images/'}
!mkdir {DATA+'images/'}

In [81]:
for i, f in enumerate(sdf_files):
    try:
        m = Chem.MolFromMolBlock(f.write('mol'))
        tmp = Compute2DCoords(m)
        Draw.MolToFile(m, DATA+'images/' + f.title + '.png')
    except KeyboardInterrupt:
        break
    except:
        print("Error encountered.")
        copy('../../bittersweet-server/app/static/images/no-image.png', 
             DATA+'images/' + sdf_files[0].title + '.png')
        
    if i % 100 == 0:
        print("Completed: %i" % i)

Completed: 0
Completed: 100
Completed: 200
Completed: 300
Completed: 400
Completed: 500
Completed: 600
Completed: 700
Completed: 800
Completed: 900
Completed: 1000
Completed: 1100
Completed: 1200
Completed: 1300
Completed: 1400
Completed: 1500
Completed: 1600
Completed: 1700
Completed: 1800
Completed: 1900
Completed: 2000
Completed: 2100
Completed: 2200
Completed: 2300
Completed: 2400
Completed: 2500
Completed: 2600
Completed: 2700
Completed: 2800
Completed: 2900
Completed: 3000
Completed: 3100
Completed: 3200
Completed: 3300
Completed: 3400
Completed: 3500
Completed: 3600
Completed: 3700
Completed: 3800
Completed: 3900
Completed: 4000
Completed: 4100
Completed: 4200
Completed: 4300
Completed: 4400
Completed: 4500
Completed: 4600
Completed: 4700
Completed: 4800
Completed: 4900
Completed: 5000
Completed: 5100
Completed: 5200
Completed: 5300
Completed: 5400
Completed: 5500
Completed: 5600
Completed: 5700
Completed: 5800
Completed: 5900
Completed: 6000
Completed: 6100
Completed: 6200
Comp

### Similarity Search

In [54]:
from pybel import Outputfile, readstring

In [56]:
chem_info = pd.read_csv(DATA+'chemical-lexicon.tsv', sep='\t')

In [67]:
largeSDfile = Outputfile("sdf", "../dietrx/static/allmol.sdf", overwrite=True)
for i, row in chem_info.iterrows():
    m = readstring("smi", row['isomeric_smiles'])
    m.title = str(row['pubchem_id'])
    m.data['pubchem_id'] = row['pubchem_id']
    
    largeSDfile.write(m)
    
largeSDfile.close()

In [69]:
%%bash
cd ../dietrx/static/
source activate bittersweetpy2
babel allmol.sdf -ofs

This will prepare an index of allmol.sdf and may take some time...
It contains 6993 molecules Estimated completion time 6.21868 seconds

 It took 6.29765 seconds
6993 molecules converted
23 audit log messages 
